In [1]:
# import libraries
#try:
  # %tensorflow_version only exists in Colab.
#  !pip install tf-nightly
#except Exception:
#  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
%pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from keras.api.models import Sequential
from keras.api.layers import Dense, Dropout
from keras.api.callbacks import EarlyStopping
from keras.api.layers import TextVectorization


print(tf.__version__)

2024-12-13 11:13:56.430757: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Note: you may need to restart the kernel to use updated packages.


/Users/knut/anaconda3/envs/experiments/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.16.2


In [2]:
# get data files
#!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
#!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
#You should create a function called predict_message that takes a message string as an argument and returns a list. 
# The first element in the list should be a number between zero and one that indicates 
# the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", 
# depending on which is most likely.



In [3]:
header = ["class", "message"]

train_file = pd.read_table(train_file_path, names=header) #See https://www.geeksforgeeks.org/how-to-convert-tab-separated-file-into-a-dataframe-using-python/

test_file = pd.read_table(test_file_path, names=header)

print(train_file.head())
print(test_file.head())

  class                                            message
0   ham  ahhhh...just woken up!had a bad dream about u ...
1   ham                           you can never do nothing
2   ham  now u sound like manky scouse boy steve,like! ...
3   ham  mum say we wan to go then go... then she can s...
4   ham  never y lei... i v lazy... got wat? dat day ü ...
  class                                            message
0   ham  i am in hospital da. . i will return home in e...
1   ham         not much, just some textin'. how bout you?
2   ham  i probably won't eat at all today. i think i'm...
3   ham  don‘t give a flying monkeys wot they think and...
4   ham                                who are you seeing?


In [4]:
#In case there should be duplicates I will drop them 
train_file = train_file.drop_duplicates()

In [5]:
train_features = train_file.copy()
train_labels = train_features.pop("class") #https://www.codecademy.com/resources/docs/pandas/dataframe/pop

test_features = test_file.copy()
test_labels = test_features.pop("class")

print("train_features length " + str(train_features.shape[0]))
print("train_labels length " + str(train_labels.shape[0]))
print("test_features length " + str(test_features.shape[0]))
print("test_labels length " + str(test_labels.shape[0]))

train_labels

train_features length 3935
train_labels length 3935
test_features length 1392
test_labels length 1392


0        ham
1        ham
2        ham
3        ham
4        ham
        ... 
4174     ham
4175     ham
4176    spam
4177    spam
4178     ham
Name: class, Length: 3935, dtype: object

In [6]:
#Finding the longest messages in order to later train a RNN network
print(train_features.message.str.len().max()) #https://blog.finxter.com/python-find-longest-string-in-a-dataframe-column/
print(test_features.message.str.len().max())

910
482


In [108]:
#Transform Labels to categorical 

# Convert categorical data to numerical data using cat.codes
#train_labels = train_labels.replace("ham", "1").replace("spam", "0").astype(int)
#test_labels = test_labels.replace("ham", "1").replace("spam", "0").astype(int)


#train_labels = train_labels.astype(int)

  #see https://saturncloud.io/blog/how-to-convert-categorical-data-to-numerical-data-with-pandas/

In [ ]:
#Eventually I found no better way to get my tensorflows model work...(2.16.2). It was always complaining over a match of datatypes

train_x_np = np.array(train_file["message"])
train_y_np = np.array(pd.factorize(train_file["class"])[0]) #See https://www.geeksforgeeks.org/python-pandas-factorize/

test_x_np = np.array(test_file["message"])
test_y_np = np.array(pd.factorize(test_file["class"])[0]) #I am accessing the returned encoded values only as I am not interested in the unique set


print("train_x_np length " + str(train_x_np.shape[0]))
print("train_y_np length " + str(train_y_np.shape[0]))
print("test_x_np length " + str(test_x_np.shape[0]))
print("test_y_np length " + str(test_y_np.shape[0]))



#Finding the longest messages in order to later train a RNN network
print(len(max(train_x_np, key=len))) #See https://blog.finxter.com/how-to-find-the-longest-string-in-a-numpy-array/
print(len(max(test_x_np, key=len))) 

train_x_np length 3935
train_y_np length 3935
test_x_np length 1392
test_y_np length 1392
910
482


In [ ]:
#See https://www.tensorflow.org/tutorials/text/text_classification_rnn
encoder = TextVectorization(
    output_mode='int',
    max_tokens=1000,
    output_sequence_length=1000,
)

encoder.adapt(train_x_np)


In [ ]:
# see https://www.tensorflow.org/text/tutorials/text_classification_rnn#create_the_model
model = keras.Sequential([
    encoder,
    keras.layers.Embedding(
        input_dim = len(encoder.get_vocabulary()),
        output_dim = 64,
        mask_zero=True,
    ),
    keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(32)),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid") #This is to get return values between 0 and 1
])


model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(1e-4), #Frankly speaking I have no clue what the meaning/effect of this one is, just took it from the tensorflow tutorial...
    metrics=["accuracy"],
)

In [41]:
history = model.fit(
    x=train_x_np.astype(object),
    y=train_y_np,
    validation_data = [test_x_np.astype(object), test_y_np],
    validation_steps=20,
    epochs=5,
)

Epoch 1/5


/Users/knut/anaconda3/envs/experiments/lib/python3.9/site-packages/keras/src/backend/tensorflow/nn.py:707: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


123/123 ━━━━━━━━━━━━━━━━━━━━ 260s 2s/step - accuracy: 0.8550 - loss: 0.6447 - val_accuracy: 0.8657 - val_loss: 0.4533
Epoch 2/5
123/123 ━━━━━━━━━━━━━━━━━━━━ 198s 2s/step - accuracy: 0.8728 - loss: 0.3890
Epoch 3/5


2024-12-14 09:51:56.730867: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/knut/anaconda3/envs/experiments/lib/python3.9/contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


123/123 ━━━━━━━━━━━━━━━━━━━━ 211s 2s/step - accuracy: 0.9547 - loss: 0.1644 - val_accuracy: 0.9648 - val_loss: 0.1099
Epoch 4/5
123/123 ━━━━━━━━━━━━━━━━━━━━ 176s 1s/step - accuracy: 0.9761 - loss: 0.0939
Epoch 5/5


2024-12-14 09:58:23.623425: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


123/123 ━━━━━━━━━━━━━━━━━━━━ 164s 1s/step - accuracy: 0.9873 - loss: 0.0609 - val_accuracy: 0.9770 - val_loss: 0.0759


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  pred = model.predict(np.array([pred_text]).astype(object))
  prob = pred[0][0]
  
  return_value = [prob, "ham" if prob <0.4 else "spam"] #See https://stackoverflow.com/questions/2802726/putting-a-simple-if-then-else-statement-on-one-line

  print(return_value)

  return return_value 

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
[0.013175111, 'ham']
[0.013175111, 'ham']


In [54]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
[0.013175111, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 437ms/step
[0.41724747, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step
[0.000136912, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
[0.9156565, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
[0.8307691, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step
[0.0021458922, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
[0.00031310323, 'ham']
You passed the challenge. Great job!
